In [1]:
import psycopg2
import pandas as pd
import datacompy
import pyodbc
import useful_functions as use

In [82]:
conn = psycopg2.connect(
    host='oneview-dev-db.cwp4vd8mllvz.us-east-1.rds.amazonaws.com',
    database='oneview_content',
    user='oneviewadmin',
    password='m3CUmKz0OEOA0Mej')

OperationalError: could not translate host name "oneview-dev-db.cwp4vd8mllvz.us-east-1.rds.amazonaws.com" to address: Unknown host


In [22]:
ov_zipcode_sql = '''
    SELECT * FROM ONEVIEW.zip_code
    '''
ov_zipcode = pd.read_sql_query(ov_zipcode_sql, conn)
ov_zipcode.head()

,id,zip_code,state,city,type,county_federal_information_processing,latitude,longitude,metropolitan_statistical_area,primary_metropolitan_statistical_area
0,00617 URB LAS PRADERAS II,00617,PR,URB LAS PRADERAS II,,72017,18.4393,66.5438,7442,7440
1,00725 URB BONNEVILLE GARDENS,00725,PR,URB BONNEVILLE GARDENS,,72025,18.2331,66.0318,7442,1310
2,00725 URB CARIBE GARDENS,00725,PR,URB CARIBE GARDENS,,72025,18.2331,66.0318,7442,1310
3,00727 URB IDAMARIS GARDENS,00727,PR,URB IDAMARIS GARDENS,,72025,18.2316,66.0504,7442,1310
4,00727 URB TURABO GARDENS,00727,PR,URB TURABO GARDENS,,72025,18.2316,66.0504,7442,1310


In [23]:
ov_msa_sql = '''
    SELECT * FROM ONEVIEW.metropolitan_statistical_area
    '''
ov_msa = pd.read_sql_query(ov_msa_sql, conn)
ov_msa.head()

,code,type,name,consolidated_metropolitan_statistical_area,population
0,0040,MSA,"ABILENE, TX",,122478
1,0060,MSA,"AGUADILLA, PR",,145484
2,0080,PMSA,"AKRON, OH",28,689435
3,0120,MSA,"ALBANY, GA",,117421
4,0160,MSA,"ALBANY-SCHENECTADY-TROY, NY",,869474


In [56]:
zips = ov_zipcode[['metropolitan_statistical_area','primary_metropolitan_statistical_area','zip_code']]

In [58]:
ov_msa[ov_msa.code=='1122']

,code,type,name,consolidated_metropolitan_statistical_area,population
40,1122,CMSA,"BOSTON-WORCESTER-LAWRENCE, MA-NH-ME-CT",07,5667225


In [60]:
mini_msa = ov_msa[['code','type','name','consolidated_metropolitan_statistical_area']]

In [70]:
test = pd.merge(mini_msa[mini_msa.type.isin(['CMSA','MSA '])],zips, left_on='code',right_on='metropolitan_statistical_area', how='right')

In [72]:
test_2 = pd.merge(mini_msa[mini_msa.type.isin(['PMSA'])],test, left_on='code',right_on='primary_metropolitan_statistical_area', how='right', suffixes=['_PMSA','_MSA'])

In [66]:
ov_phys[ov_phys.zipcode=='01010']

,medical_education_number,address_type,mailing_name,last_name,first_name,middle_name,name_suffix,preferred_address_2,preferred_address_1,city,...,race_ethnicity,census_block_group,census_division,census_group,census_region,census_suffix,census_tract,type,membership_status,has_email
1045307,01676041270,1,SHANNON E SHORT DO,SHORT,SHANNON,E,,HARRINGTON PHYSICIAN SERVICES,255 E OLD STURBRIDGE RD,BRIMFIELD,...,Unknown,1,1,6,1,02,8138,Physician,Non-Member,True
1171338,02407771675,2,ELIZABETH A SMOLA MD,SMOLA,ELIZABETH,A,,BRIMFIELD RD,1529 DUNHAMTOWN RD,BRIMFIELD,...,,1,1,6,1,02,8138,Physician,Non-Member,False
1222551,02604780759,1,JAMES A GOODWIN MD,GOODWIN,JAMES,ALAN,,,255 E OLD STURBRIDGE RD,BRIMFIELD,...,,1,1,6,1,02,8138,Physician,Non-Member,True


In [ ]:
ov_zipcode[ov_zipcode.metropolitan_statistical_area.isin(ov_msa[ov_msa.type=='MSA'].code)]

In [48]:
ov_zipcode[~ov_zipcode.metropolitan_statistical_area.isin(ov_msa[ov_msa.type=='MSA '].code)].groupby('metropolitan_statistical_area').count()

,id,zip_code,state,city,type,county_federal_information_processing,latitude,longitude,primary_metropolitan_statistical_area
metropolitan_statistical_area,,,,,,,,,
,34200,34200,34200,34200,34200,34200,34200,34200,34200
1122,1227,1227,1227,1227,1227,1227,1227,1227,1227
1602,928,928,928,928,928,928,928,928,928
1642,571,571,571,571,571,571,571,571,571
1692,521,521,521,521,521,521,521,521,521
1922,698,698,698,698,698,698,698,698,698
2082,386,386,386,386,386,386,386,386,386
2162,767,767,767,767,767,767,767,767,767
3362,584,584,584,584,584,584,584,584,584


In [51]:
ov_zipcode[ov_zipcode.metropolitan_statistical_area=='1122']

,id,zip_code,state,city,type,county_federal_information_processing,latitude,longitude,metropolitan_statistical_area,primary_metropolitan_statistical_area
3138,01001 AGAWAM,01001,MA,AGAWAM,,25013,42.0686,72.6228,1122,9240
3145,01005 BARRE,01005,MA,BARRE,,25027,42.4124,72.1101,1122,1120
3147,01008 BLANDFORD,01008,MA,BLANDFORD,,25013,42.1726,72.934,1122,9240
3148,01009 BONDSVILLE,01009,MA,BONDSVILLE,P,25013,42.2106,72.3454,1122,9240
3149,01010 BRIMFIELD,01010,MA,BRIMFIELD,,25013,42.127,72.1991,1122,9240
...,...,...,...,...,...,...,...,...,...,...
7291,06374 PLAINFIELD,06374,CT,PLAINFIELD,,9015,41.6802,71.9196,1122,9240
7295,06377 NORTH STERLING,06377,CT,NORTH STERLING,,9015,41.7171,71.8245,1122,9240
7296,06377 STERLING,06377,CT,STERLING,,9015,41.7171,71.8245,1122,9240
7309,06387 WAUREGAN,06387,CT,WAUREGAN,P,9015,41.7452,71.9113,1122,9240


In [4]:
ov_phys_sql = '''
    SELECT * FROM ONEVIEW.PHYSICIAN
    '''
ov_phys= pd.read_sql_query(ov_phys_sql, conn)
ov_phys.head()

,medical_education_number,address_type,mailing_name,last_name,first_name,middle_name,name_suffix,preferred_address_2,preferred_address_1,city,...,race_ethnicity,census_block_group,census_division,census_group,census_region,census_suffix,census_tract,type,membership_status,has_email
0,03546030724,1,LAURA K KENT MD,KENT,LAURA,KEEN,,BOX 427,622 W 168TH ST,NEW YORK,...,White,1,2,6,1,00,0251,Physician,Non-Member,False
1,03546030732,2,NIKHIL GERA MD,GERA,NIKHIL,,,,22112 RIM FIRE LN,DIAMOND BAR,...,Asian,1,9,6,4,24,4033,Physician,Non-Member,True
2,03546030741,2,CLAUDIA V FERNANDEZ MD,FERNANDEZ,CLAUDIA,V,,,5630 SW 163RD AVE,SW RANCHES,...,"Hispanic, Latino, or Spanish Origin",1,5,6,3,15,0703,Physician,Non-Member,True
3,03546030759,2,CHARMAINE K SHEN MD,SHEN,CHARMAINE,KACHIT,,,166 ALPINE TER,OAKLAND,...,"Hispanic, Latino, or Spanish Origin",2,9,6,4,00,4043,Physician,Non-Member,True
4,03546030767,2,LEO PEVZNER MD,PEVZNER,LEO,,,,27862 HOMESTEAD RD,LAGUNA NIGUEL,...,White,2,9,6,4,35,0423,Physician,Non-Member,False


In [19]:
ov_phys[(ov_phys.polo_city=='PORTLAND')]

,medical_education_number,address_type,mailing_name,last_name,first_name,middle_name,name_suffix,preferred_address_2,preferred_address_1,city,...,race_ethnicity,census_block_group,census_division,census_group,census_region,census_suffix,census_tract,type,membership_status,has_email
40,03546031194,1,JOEL A AMUNDSON MD,AMUNDSON,JOEL,AARON,,,5231 NE M L KING BLVD,PORTLAND,...,White,1,9,6,4,01,0034,Physician,Non-Member,False
152,03546040606,1,YEE-CHEEN DOUNG MD,DOUNG,YEE-CHEEN,,,,3181 SW SAM JACKSON PARK RD,PORTLAND,...,"Hispanic, Latino, or Spanish Origin",2,9,6,4,00,0058,Physician,Non-Member,True
278,03546050318,2,ALANA S GOLDEN MD,GOLDEN,ALANA,SHIRA,,,3645 SE BYBEE BLVD,PORTLAND,...,White,1,9,6,4,02,0003,Physician,Non-Member,True
317,03546050776,1,ALEJANDRO PEREZ MD,PEREZ,ALEJANDRO,,,STE 498,9427 SW BARNES RD,PORTLAND,...,"Hispanic, Latino, or Spanish Origin",3,9,6,4,01,0301,Physician,Non-Member,True
342,03546051071,2,JOHN C WILSON MD,WILSON,JOHN,CAMPBELL,,,14270 SW 100TH AVE,PORTLAND,...,White,3,9,6,4,01,0308,Physician,Renewal,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1386576,03545980826,2,JEFFREY K MOORE MD,MOORE,JEFFREY,KUHN,,,47 FORESIDE RD,CUMB FORESIDE,...,White,1,1,6,1,00,0042,Physician,Renewal,True
1386679,03545991003,1,ANUPAMA G KURUP MD,KURUP,ANUPAMA,G,,STE 261,9135 SW BARNES RD,PORTLAND,...,Asian,3,9,6,4,01,0301,Physician,Non-Member,True
1386871,03546010324,2,PAUL MARTEN MD,MARTEN,PAUL,,,,3435 NW VAUGHN ST,PORTLAND,...,"Hispanic, Latino, or Spanish Origin",1,9,6,4,00,0045,Physician,Non-Member,False
1387052,03506640228,2,ELIZABETH A SERRAGE MD,SERRAGE,ELIZABETH,A G,,,27 COLE FIELD RD,CAPE ELIZ,...,White,2,1,6,1,02,0037,Physician,Non-Member,True


In [7]:
ov_phys.state

0          NY
1          CA
2          FL
3          CA
4          CA
           ..
1387222    NY
1387223    MA
1387224    NY
1387225    NY
1387226    CT
Name: state, Length: 1387227, dtype: object

In [76]:
test_2.drop_duplicates()

,code_PMSA,type_PMSA,name_PMSA,consolidated_metropolitan_statistical_area_PMSA,code_MSA,type_MSA,name_MSA,consolidated_metropolitan_statistical_area_MSA,metropolitan_statistical_area,primary_metropolitan_statistical_area,zip_code
0,0080,PMSA,"AKRON, OH",28,1692,CMSA,"CLEVELAND-AKRON, OH",28,1692,0080,44056
2,0080,PMSA,"AKRON, OH",28,1692,CMSA,"CLEVELAND-AKRON, OH",28,1692,0080,44067
8,0080,PMSA,"AKRON, OH",28,1692,CMSA,"CLEVELAND-AKRON, OH",28,1692,0080,44087
9,0080,PMSA,"AKRON, OH",28,1692,CMSA,"CLEVELAND-AKRON, OH",28,1692,0080,44201
10,0080,PMSA,"AKRON, OH",28,1692,CMSA,"CLEVELAND-AKRON, OH",28,1692,0080,44202
...,...,...,...,...,...,...,...,...,...,...,...
80422,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,99926
80423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,99927
80424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,99928
80425,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,99929


In [80]:
test_2.drop_duplicates().to_csv('../../Data/MasterfileCore/zipcode_metropolitan_statistical_areas.csv', index=False)

In [79]:
ov_zipcode.drop_duplicates()

,id,zip_code,state,city,type,county_federal_information_processing,latitude,longitude,metropolitan_statistical_area,primary_metropolitan_statistical_area
0,00617 URB LAS PRADERAS II,00617,PR,URB LAS PRADERAS II,,72017,18.4393,66.5438,7442,7440
1,00725 URB BONNEVILLE GARDENS,00725,PR,URB BONNEVILLE GARDENS,,72025,18.2331,66.0318,7442,1310
2,00725 URB CARIBE GARDENS,00725,PR,URB CARIBE GARDENS,,72025,18.2331,66.0318,7442,1310
3,00727 URB IDAMARIS GARDENS,00727,PR,URB IDAMARIS GARDENS,,72025,18.2316,66.0504,7442,1310
4,00727 URB TURABO GARDENS,00727,PR,URB TURABO GARDENS,,72025,18.2316,66.0504,7442,1310
...,...,...,...,...,...,...,...,...,...,...
80424,99928 WARD COVE,99928,AK,WARD COVE,P,2130,55.4117,131.7252,,
80425,99929 WRANGELL,99929,AK,WRANGELL,P,2275,56.4717,132.3872,,
80426,99950 EDNA BAY,99950,AK,EDNA BAY,P,2130,55.7819,133.0759,,
80427,99950 KASAAN,99950,AK,KASAAN,P,2130,55.7819,133.0759,,
